In [13]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import os

In [14]:
def calculate_recent_form(df, window=5):
    df['HomeForm'] = df.groupby('HomeTeam')['HomePoints'].rolling(window).sum().reset_index(level=0, drop=True)
    df['AwayForm'] = df.groupby('AwayTeam')['AwayPoints'].rolling(window).sum().reset_index(level=0, drop=True)
    return df
def calculate_recent_home_away_form(df, window=5):
    df['HomeRecentHomeForm'] = df.groupby('HomeTeam')['HomePoints'].rolling(window).sum().reset_index(level=0, drop=True)
    df['AwayRecentAwayForm'] = df.groupby('AwayTeam')['AwayPoints'].rolling(window).sum().reset_index(level=0, drop=True)
    return df

def calculate_elo(df, k=20, home_advantage=100):
    teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()
    elo_ratings = {team: 1500 for team in teams}  # inizializza tutti i team con 1500 punti Elo

    home_elo, away_elo = [], []
    for index, row in df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']

        # Get current Elo ratings
        home_team_elo = elo_ratings[home_team] + home_advantage
        away_team_elo = elo_ratings[away_team]

        # Calculate expected result
        expected_home = 1 / (1 + 10**((away_team_elo - home_team_elo) / 400))
        expected_away = 1 - expected_home

        # Update Elo ratings based on match result
        if row['FTHG'] > row['FTAG']:  # home win
            home_points = 1
            away_points = 0
        elif row['FTHG'] < row['FTAG']:  # away win
            home_points = 0
            away_points = 1
        else:  # draw
            home_points = away_points = 0.5

        elo_ratings[home_team] += k * (home_points - expected_home)
        elo_ratings[away_team] += k * (away_points - expected_away)

        # Store Elo for this match
        home_elo.append(elo_ratings[home_team])
        away_elo.append(elo_ratings[away_team])

    df['HomeEloRating'] = home_elo
    df['AwayEloRating'] = away_elo
    return df

In [15]:
# Specifica il percorso della cartella
folder_path = 'data/League1'
# folder_path = 'data/SerieB'

# Itera su tutti i file nella cartella
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
        # Verifica se il percorso è un file (e non una cartella)
    if os.path.isfile(file_path):
        print(f"Processing file: {filename}")

    data = pd.read_excel(file_path)
    # Supponiamo che il DataFrame si chiami data
    # Aggiungere le colonne per i goal cumulativi
    data['HomeGoalsCumulative'] = 0
    data['AwayGoalsCumulative'] = 0

    # Creare un dizionario per tenere traccia dei goal cumulativi di ogni squadra
    goals_cumulative = {}

    # Iterare sulle righe del DataFrame
    for index, row in data.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        home_goals = row['FTHG']
        away_goals = row['FTAG']
        
        # Inizializzare il conteggio dei goal per le squadre se non già presente
        if home_team not in goals_cumulative:
            goals_cumulative[home_team] = 0
        if away_team not in goals_cumulative:
            goals_cumulative[away_team] = 0
        
        # Assegnare i goal cumulativi fino a quel momento
        data.at[index, 'HomeGoalsCumulative'] = goals_cumulative[home_team]
        data.at[index, 'AwayGoalsCumulative'] = goals_cumulative[away_team]
        
        # Aggiornare i goal cumulativi con i goal della partita attuale
        goals_cumulative[home_team] += home_goals
        goals_cumulative[away_team] += away_goals

    import pandas as pd

    # Supponiamo che il DataFrame si chiami data
    # Aggiungere le colonne per i punti cumulativi
    data['HomePointsCumulative'] = 0
    data['AwayPointsCumulative'] = 0

    # Creare due dizionari per tenere traccia dei punti cumulativi di ogni squadra
    points_cumulative = {}

    # Iterare sulle righe del DataFrame
    for index, row in data.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        result = row['FTR']
        
        # Inizializzare i punti per le squadre se non già presenti
        if home_team not in points_cumulative:
            points_cumulative[home_team] = 0
        if away_team not in points_cumulative:
            points_cumulative[away_team] = 0
        
        # Assegnare i punti cumulativi fino a quel momento
        data.at[index, 'HomePointsCumulative'] = points_cumulative[home_team]
        data.at[index, 'AwayPointsCumulative'] = points_cumulative[away_team]
        
        # Aggiornare i punti cumulativi in base al risultato della partita
        if result == 'H':  # Vittoria della squadra di casa
            points_cumulative[home_team] += 3
        elif result == 'A':  # Vittoria della squadra ospite
            points_cumulative[away_team] += 3
        elif result == 'D':  # Pareggio
            points_cumulative[home_team] += 1
            points_cumulative[away_team] += 1

    # Supponiamo che il DataFrame si chiami data
    # Aggiungere le colonne per i goal subiti cumulativi
    data['HomeGoalsConcededCumulative'] = 0
    data['AwayGoalsConcededCumulative'] = 0

    # Creare un dizionario per tenere traccia dei goal subiti cumulativi di ogni squadra
    goals_conceded_cumulative = {}

    # Iterare sulle righe del DataFrame
    for index, row in data.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        home_goals = row['FTHG']  # Goal fatti dalla squadra di casa
        away_goals = row['FTAG']  # Goal fatti dalla squadra ospite
        
        # Inizializzare i goal subiti per le squadre se non già presenti
        if home_team not in goals_conceded_cumulative:
            goals_conceded_cumulative[home_team] = 0
        if away_team not in goals_conceded_cumulative:
            goals_conceded_cumulative[away_team] = 0
        
        # Assegnare i goal subiti cumulativi fino a quel momento
        data.at[index, 'HomeGoalsConcededCumulative'] = goals_conceded_cumulative[home_team]
        data.at[index, 'AwayGoalsConcededCumulative'] = goals_conceded_cumulative[away_team]
        
        # Aggiornare i goal subiti cumulativi con i goal della partita attuale
        goals_conceded_cumulative[home_team] += away_goals  # La squadra di casa subisce i goal della squadra ospite
        goals_conceded_cumulative[away_team] += home_goals  # La squadra ospite subisce i goal della squadra di casa

    data['MatchGoal'] = data['FTHG'] + data['FTAG']

    # Calcolo della differenza di punti tra le due squadre
    data['PointsDifference'] = abs(data['HomePointsCumulative'] - data['AwayPointsCumulative'])

    # Calcolo del rapporto tra i goal fatti e subiti per la squadra di casa
    data['HomeGoalsRatio'] = data['HomeGoalsCumulative'] / data['HomeGoalsConcededCumulative']

    # Calcolo del rapporto tra i goal fatti e subiti per la squadra in trasferta
    data['AwayGoalsRatio'] = data['AwayGoalsCumulative'] / data['AwayGoalsConcededCumulative']

    # Calcolo della differenza tra i goal fatti della squadra di casa rispetto a quelli della squadra in trasferta
    data['GoalsDifference'] = abs(data['HomeGoalsCumulative'] - data['AwayGoalsCumulative'])

    # Calcolo della differenza tra i goal subiti dalla squadra di casa rispetto a quelli subiti dalla squadra in trasferta
    data['ConcededGoalsDifference'] = abs(data['HomeGoalsConcededCumulative'] - data['AwayGoalsConcededCumulative'])

    data['HomePoints'] = data.apply(lambda x: 3 if x['FTHG'] > x['FTAG'] else (1 if x['FTHG'] == x['FTAG'] else 0), axis=1)
    data['AwayPoints'] = data.apply(lambda x: 3 if x['FTAG'] > x['FTHG'] else (1 if x['FTHG'] == x['FTAG'] else 0), axis=1)

    data = calculate_recent_form(data)
    data = calculate_recent_home_away_form(data)
    data = calculate_elo(data)

    resultFileName = filename[:-5] + '-E.xlsx'
    data.to_excel(resultFileName, index=False)

Processing file: 19-20.xlsx
Processing file: 20-21.xlsx
Processing file: 21-22.xlsx
Processing file: 22-23.xlsx
Processing file: 23-24.xlsx


## TODO
- Average Goal Per Match H/A
